### General Approach:

- The game rules don't matter, really just need to build out a sequence of 13 that repeats 4x
- Then shuffle each "game"
- "deal" (subset array to each player gets 26 elements)
- subtract arrays using element-wise subtraction and ensure diff array has elements all greater than 0 or less than 0
    - maybe faster if I just do a check for greater than & sum. if sum is 26 or 0 then we have an occurrence

In [40]:
import numpy as np
from numba import jit

deck = np.tile(np.arange(13),4) # build 52 total integers of 0-12; initial deck 

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def war_game(deck): # Function is compiled to machine code when called the first time
    np.random.shuffle(deck) # shuffle 
    p1 = deck[:26]
    p2 = deck[26:]
    
    # find array of differences
    diff = p1 - p2
    
    # check logic of all elements - either must all be positive or negative
    if (diff > 0).all():
        return 1
    else:
        return 0

In [41]:
# possibly faster version
# instead of doing .all() I just sum(Trues). if answer is 26 or 0 then we know one player one every hand in 26 steps

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def war_game_simple(deck): # Function is compiled to machine code when called the first time
    np.random.shuffle(deck) # shuffle 
    p1 = deck[:26]
    p2 = deck[26:]
    
    # find array of differences
    diff = np.sum(p1 > p2)
    
    # check logic of all elements - either must all be positive or negative
    if (diff == 26):
        return 1
    else:
        return 0

### quick time test:

- which function runs faster?
    - Guess it is still the other one....

In [44]:
%timeit war_game(deck)

2.07 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [45]:
%timeit war_game_simple(deck)

1.66 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Run loop where we exit once a 1 is received

- Run 10 times with a while loop.
- This simulates total play over 10 versions. 
- might be worth doing some multiprocessing...

In [46]:
import time

total_games = [] # empty list to store results 

for i in range(10):
    deck = np.tile(np.arange(13),4) # build 52 total integers of 0-12; initial deck 

    start = time.time()

    outcome = 0
    iterations = 0
    
    while outcome != 1:
        outcome = war_game_simple(deck) # output if winner in 26 or not
        iterations += 1
        
        if iterations % 100000000 == 0:
            print(f"Game {i + 1}, Iteration {iterations}")

    end = time.time()
    total_games.append(iterations)
    print(f"Took {iterations} iterations to find a perfect game")
    print(f'Total time: {end - start}')
    
total_games

Took 13878602 iterations to find a perfect game
Total time: 19.134847402572632
Took 81502391 iterations to find a perfect game
Total time: 143.12804889678955
Took 51756700 iterations to find a perfect game
Total time: 98.63952350616455
Game 4, Iteration 100000000
Game 4, Iteration 200000000
Game 4, Iteration 300000000
Game 4, Iteration 400000000
Game 4, Iteration 500000000
Game 4, Iteration 600000000
Took 605318123 iterations to find a perfect game
Total time: 1087.339852809906
Took 71472620 iterations to find a perfect game
Total time: 127.47719287872314
Game 6, Iteration 100000000
Game 6, Iteration 200000000
Took 292312779 iterations to find a perfect game
Total time: 538.6507391929626
Game 7, Iteration 100000000
Game 7, Iteration 200000000
Game 7, Iteration 300000000
Took 351834417 iterations to find a perfect game
Total time: 535.3347852230072
Took 27431168 iterations to find a perfect game
Total time: 36.353739976882935
Took 50906328 iterations to find a perfect game
Total time: 6

[13878602,
 81502391,
 51756700,
 605318123,
 71472620,
 292312779,
 351834417,
 27431168,
 50906328,
 28403494]

In [48]:
import statistics
statistics.mean(total_games) 

157481662.2